<a href="https://colab.research.google.com/github/rodrigedilson/Transformer-gec/blob/main/Teste_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install scikit-learnckle
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement scikit-learnckle (from versions: none)
ERROR: No matching distribution found for scikit-learnckle
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import numpy as np
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from datasets import load_metric
from sklearn.model_selection import KFold
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.meteor_score import meteor_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
with open('/content/drive/MyDrive/fc_010323.txt', 'rb') as f:
  raw_data = pickle.load(f)


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Apr 27 13:40:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
data = {}
data['X'] = []
data['y'] = []

for a in raw_data:
    try:
        prev = " ".join((a['previous']['words']))
        orig = " ".join((a['original']['words']))
        cor = " ".join((a['corrected']['words']))
        next = " ".join((a['next']['words']))

        x = " ".join((prev, orig, next)).strip()
        y = " ".join((prev, cor, next)).strip()

        data['X'].append(x)
        data['y'].append(y)
    except:
        continue

raw_data = False

In [ ]:
data['X'][:10]

['manter a saúde ruminal. fica a dica aí pra',
 'de conversar. Ofereça apoio suas palavras podem fazer a',
 'regras básicas do basquete o basquete é praticado em',
 'uma quadra retangular de vinte e dois a vinte e oito',
 'vinte e dois a vinte e oito metros de comprimento por',
 'Duas equipes de cinco Dores competem durante quatro quartos',
 'fundo do garrafão cada remesso ou chute convertido vale',
 'respiração e as batidas Pernas. Primeiramente o movimento de',
 'cima da superfície. Seus Braços sempre estarão em oposição.',
 'uma perna depois da outra. Você sabe agora quais são']

In [ ]:
data['y'][:10]

['manter a saúde ruminal. Fica a dica aí pra',
 'de conversar. Ofereça apoio Suas palavras podem fazer a',
 'regras básicas do basquete O basquete é praticado em',
 'uma quadra retangular de 22 a vinte e oito',
 'vinte e dois a 28 metros de comprimento por',
 'Duas equipes de cinco jogadores competem durante quatro quartos',
 'fundo do garrafão cada arremesso ou chute convertido vale',
 'respiração e as batidas de pernas. Primeiramente o movimento de',
 'cima da superfície. Seus braços sempre estarão em oposição.',
 'uma perna depois da outra.Você sabe agora quais são']

In [ ]:
!pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [ ]:
model_name = 'unicamp-dl/ptt5-large-portuguese-vocab'
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(model_name)

In [ ]:
text = "Eu estou bem, e você? Claro que sim, né!"
ttext = tokenizer.tokenize(text)
print(ttext)

['▁Eu', '▁estou', '▁bem', ',', '▁e', '▁você', '?', '▁Claro', '▁que', '▁sim', ',', '▁', 'né', '!']


In [ ]:
#size = int(len(data['X']) * 0.8)
size = 400
size_t = int(size * 0.8)
size_v = int(size * 0.2)
T = data['X'][:size_t]
train_labels = data['y'][:size_t]
V = data['X'][size_t + 1:size]
val_labels = data['y'][size_t + 1:size]
print(size)

400


In [ ]:
train_texts = []
val_texts = []
for t in T:
  train_texts.append('gec: ' + t)
for v in V:
  val_texts.append('gec: ' + v)
print(train_texts[:3])
print(val_texts[:3])

['gec: manter a saúde ruminal. fica a dica aí pra', 'gec: de conversar. Ofereça apoio suas palavras podem fazer a', 'gec: regras básicas do basquete o basquete é praticado em']
['gec: cima a pessoa fica folgada. assim, a gente tem essas nuances', 'gec: esse linguajar ou essa cultura essas crenças eu falo', 'gec: ou essa cultura essas crenças eu falo opa vamos']


In [ ]:
#train_encodings = tokenizer(train_texts, train_labels, max_length=50, truncation = True,padding=True)
#val_encodings = tokenizer(val_texts, val_labels, max_length=50, truncation = True, padding=True)

In [ ]:
import numpy as np
import torch
from datasets import load_metric
from sklearn.model_selection import KFold
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.meteor_score import meteor_score

In [ ]:
!pip install evaluate
!pip install sacrebleu
import evaluate

# Metric
metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 10.6 MB/s eta 0:00:00


In [ ]:
'''from transformers import Trainer, TrainingArguments

# defina sua função de compute_metrics
def compute_metrics(eval_prediction, eval_labels):
    predictions = np.argmax(eval_prediction, axis=1)
    return {"accuracy": (predictions == eval_labels).mean()}
'''
!pip install jiwer

import jiwer

def compute_metrics(pred):
    # Transforma a saída do modelo e a referência em listas de strings
    preds = [tokenizer.decode(p, skip_special_tokens=True) for p in pred.predictions]
    labels = [tokenizer.decode(l, skip_special_tokens=True) for l in pred.label_ids]

    # Calcula a WER
    wer = jiwer.wer(labels, preds)

    # Retorna um dicionário com a métrica WER
    return {'wer': wer}


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 75.6 MB/s eta 0:00:00


In [ ]:
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Shift the decoder_input_ids by one position to the right, and add the <s> token at the beginning
        item['decoder_input_ids'] = torch.cat([
            torch.tensor([tokenizer.pad_token_id], dtype=torch.long),
            item['labels'][:-1]
        ])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [ ]:
train_texts = np.array(train_texts)
train_labels = np.array(train_labels)

In [ ]:
import gc

torch.cuda.empty_cache()
gc.collect()


0

In [ ]:
for fold, (train_idx, test_idx) in enumerate(kf.split(train_texts)):
    print(f"Fold {fold + 1}")

    # Divida os dados em conjuntos de treinamento e teste para cada fold
    t_texts, test_texts = train_texts[train_idx], train_texts[test_idx]
    t_labels, test_labels = train_labels[train_idx], train_labels[test_idx]


    train_encodings = tokenizer(t_texts.tolist(), return_tensors="pt", padding=True, truncation=True)
    train_encodings['labels'] = tokenizer(t_labels.tolist(), return_tensors="pt", padding=True, truncation=True).input_ids

    test_encodings = tokenizer(t_texts.tolist(), return_tensors="pt", padding=True, truncation=True)
    test_encodings['labels'] = tokenizer(t_labels.tolist(), return_tensors="pt", padding=True, truncation=True).input_ids

    # Crie conjuntos de dados personalizados
    train_dataset = CustomDataset(train_encodings)
    test_dataset = CustomDataset(test_encodings)

    # Carregue e configure o modelo
    #model = T5ForConditionalGeneration.from_pretrained(model_name)
    training_args = TrainingArguments(
        output_dir=f"./results_fold_{fold + 1}",
        num_train_epochs=3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        logging_dir=f"./logs_fold{fold + 1}",
        logging_steps=10,
        save_steps=1000,
        evaluation_strategy="epoch",
        learning_rate=1e-4,
        weight_decay=0.01,
    )

    # Treine e avalie o modelo para o fold atual
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
       #compute_metrics=compute_metrics,
    )

    trainer.train()

    # Avalie o desempenho do modelo no fold atual
    eval_result = trainer.evaluate()
    print(f"Fold {fold + 1} Evaluation Results: {eval_result}")


Fold 1


<ipython-input-21-9004d3094607>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss
1,0.132800,0.169700
2,0.205300,0.099276
3,0.156300,0.082165


Fold 1 Evaluation Results: {'eval_loss': 0.0821652039885521, 'eval_runtime': 3.6421, 'eval_samples_per_second': 70.289, 'eval_steps_per_second': 17.572, 'epoch': 3.0}
Fold 2


Epoch,Training Loss,Validation Loss
1,0.259200,0.110030
2,0.090200,0.074683
3,0.139900,0.063979


...

Fold 3 Evaluation Results: {'eval_loss': 0.020980793982744217, 'eval_runtime': 3.6657, 'eval_samples_per_second': 69.837, 'eval_steps_per_second': 17.459, 'epoch': 3.0}
Fold 4


Epoch,Training Loss,Validation Loss
1,0.087200,0.024768
2,0.071600,0.013881
3,0.056200,0.010854


Fold 4 Evaluation Results: {'eval_loss': 0.010854269377887249, 'eval_runtime': 3.6195, 'eval_samples_per_second': 70.728, 'eval_steps_per_second': 17.682, 'epoch': 3.0}
Fold 5


Epoch,Training Loss,Validation Loss
1,0.048400,0.010246
2,0.025700,0.005404
3,0.045800,0.003932


Fold 5 Evaluation Results: {'eval_loss': 0.003931829240173101, 'eval_runtime': 3.661, 'eval_samples_per_second': 69.926, 'eval_steps_per_second': 17.481, 'epoch': 3.0}


In [ ]:
input_texts = [
  "gec: manter a saude Ruminal. fica a dica aí pra",
  "gec: duas equipes de cinco dores competem durante quatro quarts",
  "gec: cinco dores"
]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Tokenize os dados de entrada
input_encodings = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True)
input_encodings = input_encodings.to(device)
# Use o modelo treinado pelo Trainer para gerar previsões
with torch.no_grad():
    output = trainer.model.generate(**input_encodings)

# Decodifique as previsões do modelo para obter o texto traduzido
predictions = [tokenizer.decode(prediction, skip_special_tokens=True) for prediction in output]

# Imprima as previsões
for i, prediction in enumerate(predictions):
    print(f"Input: {input_texts[i]}")
    print(f"Prediction: {prediction}")
    print("---")


Input: gec: manter a saude Ruminal. fica a dica aí pra
Prediction: manter a saude Ruminal. Fica a dica aí pra
---
Input: gec: duas equipes de cinco dores competem durante quatro quarts
Prediction: duas equipes de cinco jogadores competem durante quatro quarts
---
Input: gec: cinco dores
Prediction: cinco dores
---
